In [20]:
from langchain.chat_models import ChatOpenAI
from langchain.tools import tool
from langchain.prompts import ChatPromptTemplate
from langchain.schema.agent import AgentFinish
from langchain.tools.render import format_tool_to_openai_function
from langchain.agents.output_parsers import OpenAIFunctionsAgentOutputParser
from pydantic import BaseModel, Field
from langchain.prompts import MessagesPlaceholder
from typing import List, Optional
import pandas as pd

In [21]:
class movieDetails(BaseModel):
    movie_genre :List[str] = Field(description="The genres of the movies/movie you like or you have watched before seperated by comma")
    movies_watched: Optional[List[str]] = Field(description="The names of the movies you have watched before or like, separated by comma")

class booksDetails(BaseModel):
    books_genre :List[str] = Field(description="The genres of the Books/Book you like or you have read before seperated by comma")
    books_read: Optional[List[str]] = Field(description="The names of the Book/books you have read before or like, separated by comma")


In [22]:
@tool(args_schema=movieDetails)
def get_movies_you_might_love(movieDetails: movieDetails) -> dict:
    """Recommends movies based on genres or previously watched movies."""

    movie_df = pd.read_csv("./imdb_top_1000.csv")
   
    if movieDetails.movie_genre and len(movieDetails.movie_genre) > 0:
        filtered_movies = filter_items_by_genre_or_title(
            movie_df,
            genre_column='Genre',
            title_column='Series_Title',
            genres=movieDetails.movie_genre,
            titles=movieDetails.movies_watched
        )
        return {"Movie you might like": list(filtered_movies['Series_Title'])}
    else:
        return {"MovieError": "Please provide either a genre or a list of movies you've watched."}

In [16]:
@tool(args_schema=booksDetails)
def get_books_you_might_love(booksDetails: booksDetails) -> dict:
    """Recommends books based on genres or previously read books."""
    
    books_df = pd.read_csv("./BooksDatasetClean.csv")

    if booksDetails.books_genre or booksDetails.books_read:
        filtered_books = filter_items_by_genre_or_title(
            books_df,
            genre_column='Category',
            title_column='Title',
            genres=booksDetails.books_genre,
            titles=booksDetails.books_read
        )
        return {"books you might like": list(filtered_books['Title'])}
    else:
        return {"BookError": "Please provide either a genre or a list of books you've read."}


In [23]:
def filter_items_by_genre_or_title(df: pd.DataFrame, genre_column: str, title_column: str, genres: List[str] = None, titles: List[str] = None) -> pd.DataFrame:
    """Helper function to filter items by genre or title.""" 
    if genres and len(genres) > 0:
        genres_you_like = [genre.strip().lower() for genre in genres]
        genre_filter = df[genre_column].str.contains('|'.join(genres_you_like), case=False, na=False)
        df = df[genre_filter]

    if titles and len(titles) > 0:
        titles = [title.strip().lower() for title in titles]
        title_filter = df[title_column].str.contains('|'.join(titles), case=False, na=False)
        df = df[title_filter]

    return df

In [19]:
tools = [
     get_movies_you_might_love, get_books_you_might_love
]
func_tools = [format_tool_to_openai_function(i) for i in tools]

C:\Users\Design Dep\AppData\Local\Temp\ipykernel_17392\2668362382.py:4: LangChainDeprecationWarning: The function `format_tool_to_openai_function` was deprecated in LangChain 0.1.16 and will be removed in 1.0. Use :meth:`~langchain_core.utils.function_calling.convert_to_openai_function()` instead.
  func_tools = [format_tool_to_openai_function(i) for i in tools]


In [24]:
model = ChatOpenAI(temperature=0, model="chat-gpt-4.0").bind(functions=func_tools)

C:\Users\Design Dep\AppData\Local\Temp\ipykernel_17392\1092935690.py:1: LangChainDeprecationWarning: The class `ChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import ChatOpenAI``.
  model = ChatOpenAI(temperature=0, model="chat-gpt-4.0").bind(functions=func_tools)


In [ ]:
prompt = ChatPromptTemplate.from_messages([
        ("")
    ])